# Understanding TemporalInstanton.jl

By now I have written a fair bit of code for performing temporal instanton analysis. Any modifications I make ripple through the codebase, causing widespread problems. For my own benefit, it's time to understand the code better and document it thoroughly. Most of this documentation should be embedded in the Julia source itself, taking advantage of [Julia's excellent documentation setup][1].

Some of the documentation doesn't fit in the source itself. Describing each source file, for example, requires external treatment.

[1]: http://docs.julialang.org/en/latest/manual/documentation/

## Files

### Temporal Instanton.jl

The module around which everything else is built. This file loads all other source files and exports useful functions when it is called. All temporal instanton analysis may be kicked off with

```julia
include("/path/to/TemporalInstanton.jl") # tell Julia about the module
using TemporalInstanton # load and export all functions
```

### DataLoad.jl

All code defining custom types and implementing data import stuff is here. There are two types.

* `InstantonInputData` has a field for each parameter necessary to begin instanton analysis. This type is a container that makes it easy to gather all pertinent data under a single name and pass it to the main analysis function.

* `InstantonOutputData` contains all data produced by temporal instanton analysis. With these two custom types defined, analysis can be performed as follows:


1. Gather all input data into an instance `d` of `InstantonInputData`.
2. Call `solve_temporal_instanton(d)` to perform analysis.
3. Study structured output data `o`, which is an instance of `InstantonOutputData`.

Step 1 requires loading data stored on the hard disk and generating reasonable values to fill in gaps. The functions in `DataLoad.jl` handle these tasks.

* `load_rts_96_data()` loads data from Jennifer Marley's OPF studies. This is a special case I began using while working on instantaneous analysis. I keep it around as a constant scenario against which I can test performance. I am, however, thinking of retiring it. The biggest reasons for keeping it around --that I could compare results with Jenny, and that I had no other way to build test scenarios-- are no longer significant.

* `mat2tmpinst()` is the workhorse that takes the name of a Matpower case, fills in gaps, and returns `d::InstantonInputData` ready for analysis. Filling in the gaps necessitates a couple assumptions:

    1. All generators whose output is smaller than the mean are converted into wind farms. This tends to be a good way to distribute wind throughout the network without disturbing things too much, but can cause trouble in networks that have only a few small generators.
    2. All line conductors are assumed to be Waxwing ACSR. Every line has the same thermal parameters regardless of its voltage. It would be nice to do away with this assumption by using Jon's conductor assignment code, which uses electric parameters as features to predict reasonable thermal parameters.

* `createY()` generates an admittance matrix. Pretty straightforward.

* `return_line_conductors()` was a naive attempt to use line voltage to assign thermal parameters. I was trying to support both Waxwing and Dove to emulate the approach in Mads's RTS-96 case study.

### PowerFlow.jl

### ThermalModel.jl

### QCQPMatrixBuilding.jl

### manipulations.jl

### SolveSecular.jl

### plot.jl

### solvetmpinst.jl

In [1]:
# Using one core
include("../src/TemporalInstanton.jl")
using TemporalInstanton

In [ ]:
# Using multiple cores
addprocs(2) # vary number of concurrent processes here
@everywhere include("../src/TemporalInstanton.jl")
# @everywhere include("../src/mat2tmpinst.jl")
@everywhere using TemporalInstanton

In [3]:
# test case used for timing analysis
d = testcase("timing")
@time o = solve_temporal_instanton(d);

sort(o.score)

r=0 check: 	removing 1 lines
ISF pre-check: 	removing lines Int64[]
  

104-element Array{Tuple{Float64,Int64},1}:
 (0.263787830020006,25)  
 (0.3273164183285288,30) 
 (1.2886107366327355,69) 
 (2.2041515936832066,89) 
 (2.322973886040895,118) 
 (2.42316154712363,64)   
 (4.67875546231252,42)   
 (4.865240811106734,102) 
 (4.899167201158152,31)  
 (5.678266048785588,80)  
 (5.938620037616521,1)   
 (6.439172990446961,100) 
 (6.4608887584718095,62) 
 ⋮                       
 (178.0855817862111,99)  
 (303.00328573243416,103)
 (370.61443512027347,66) 
 (370.61443512027347,67) 
 (391.3070873721824,33)  
 (428.94449223314234,40) 
 (923.97998538764,72)    
 (979.1988880469704,117) 
 (1319.8524187115077,79) 
 (1390.0270570261027,110)
 (1967.2698882361206,73) 
 (1967.2698882361206,74) 

8.445260 seconds (8.65 M allocations: 1.197 GB, 2.15% gc time)
